# Goal:

Try training LGBM with basic features (14d prediction)

Group features:
- same id, product id, name first, name second, categores, warehouse, warhouse location, combos
- all of these --> lag sales, rolling sales, etc.

Date features:
- dow, month, year, etc.
- is holiday